# Open Source LLM serving using the Azure ML Python SDK

[Note] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
lab_prep_dir = os.getcwd().split("slm-innovator-lab")[0] + "slm-innovator-lab/0_lab_preparation"
sys.path.append(os.path.abspath(lab_prep_dir))

from common import check_kernel
check_kernel()

In [ ]:
%store -r job_name
try:
    job_name
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the previous notebook (model training) again.")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

## 1. Load config file

---


In [ ]:
import os
import yaml
from logger import logger
from datetime import datetime

snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open("config.yml") as f:
    d = yaml.load(f, Loader=yaml.FullLoader)

AZURE_SUBSCRIPTION_ID = d["config"]["AZURE_SUBSCRIPTION_ID"]
AZURE_RESOURCE_GROUP = d["config"]["AZURE_RESOURCE_GROUP"]
AZURE_WORKSPACE = d["config"]["AZURE_WORKSPACE"]
AZURE_DATA_NAME = d["config"]["AZURE_DATA_NAME"]
DATA_DIR = d["config"]["DATA_DIR"]
CLOUD_DIR = d["config"]["CLOUD_DIR"]
HF_MODEL_NAME_OR_PATH = d["config"]["HF_MODEL_NAME_OR_PATH"]
IS_DEBUG = d["config"]["IS_DEBUG"]

azure_env_name = d["serve"]["azure_env_name"]
azure_model_name = d["serve"]["azure_model_name"]
azure_endpoint_name = d["serve"]["azure_endpoint_name"]
azure_deployment_name = d["serve"]["azure_deployment_name"]
azure_serving_cluster_size = d["serve"]["azure_serving_cluster_size"]


logger.info("===== 0. Azure ML Deployment Info =====")
logger.info(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
logger.info(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
logger.info(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
logger.info(f"AZURE_DATA_NAME={AZURE_DATA_NAME}")
logger.info(f"DATA_DIR={DATA_DIR}")
logger.info(f"CLOUD_DIR={CLOUD_DIR}")
logger.info(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
logger.info(f"IS_DEBUG={IS_DEBUG}")

logger.info(f"azure_env_name={azure_env_name}")
logger.info(f"azure_model_name={azure_model_name}")
logger.info(f"azure_endpoint_name={azure_endpoint_name}")
logger.info(f"azure_deployment_name={azure_deployment_name}")
logger.info(f"azure_serving_cluster_size={azure_serving_cluster_size}")

<br>

## 2. Serving preparation

---

### 2.1. Configure workspace details

To connect to a workspace, we need identifying parameters - a subscription, a resource group, and a workspace name. We will use these details in the MLClient from azure.ai.ml to get a handle on the Azure Machine Learning workspace we need. We will use the default Azure authentication for this hands-on.


In [ ]:
# import required libraries
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

logger.info(f"===== 2. Serving preparation =====")
logger.info(f"Calling DefaultAzureCredential.")
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE
)

### 2.2. Create model asset


In [ ]:
def get_or_create_model_asset(
    ml_client,
    model_name,
    job_name=None,
    model_dir="outputs",
    model_type="custom_model",
    update=False,
):
    try:
        latest_model_version = max(
            [int(m.version) for m in ml_client.models.list(name=model_name)]
        )
        if update:
            raise ResourceExistsError("Found Model asset, but will update the Model.")
        else:
            model_asset = ml_client.models.get(
                name=model_name, version=latest_model_version
            )
            logger.info(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        logger.info(f"Exception: {e}")
        if job_name is None:
            model_path = model_dir
        else:
            model_path = (
                f"azureml://jobs/{job_name}/outputs/artifacts/paths/{model_dir}/"
            )
        run_model = Model(
            name=model_name,
            path=model_path,
            description="Model created from run.",
            type=model_type,  # mlflow_model, custom_model, triton_model
        )
        model_asset = ml_client.models.create_or_update(run_model)
        logger.info(f"Created Model asset: {model_name}")

    return model_asset

In [ ]:
model_dir = d["train"]["model_dir"]
model = get_or_create_model_asset(
    ml_client,
    azure_model_name,
    job_name,
    model_dir,
    model_type="custom_model",
    update=False,
)

### 2.3. Create AzureML environment

Azure ML defines containers (called environment asset) in which your code will run. We can use the built-in environment or build a custom environment (Docker container, conda). This hands-on uses Docker container.


#### Docker environment


In [ ]:
from azure.ai.ml.entities import Environment, BuildContext


def get_or_create_docker_environment_asset(
    ml_client,
    env_name,
    docker_dir,
    dockerfile_path="Dockerfile",
    inference_config=None,
    update=False,
):

    try:
        latest_env_version = max(
            [int(e.version) for e in ml_client.environments.list(name=env_name)]
        )
        if update:
            raise ResourceExistsError(
                "Found Environment asset, but will update the Environment."
            )
        else:
            env_asset = ml_client.environments.get(
                name=env_name, version=latest_env_version
            )
            logger.info(f"Found Environment asset: {env_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        logger.info(f"Exception: {e}")
        env_docker_image = Environment(
            build=BuildContext(dockerfile_path=dockerfile_path, path=docker_dir),
            name=env_name,
            description="Environment created from a Docker context.",
            inference_config=inference_config,
        )
        env_asset = ml_client.environments.create_or_update(env_docker_image)
        logger.info(f"Created Environment asset: {env_name}")

    return env_asset


inference_config = {
    "liveness_route": {
        "port": 8000,
        "path": "/health",
    },
    "readiness_route": {
        "port": 8000,
        "path": "/health",
    },
    "scoring_route": {
        "port": 8000,
        "path": "/",
    },
}

env = get_or_create_docker_environment_asset(
    ml_client,
    azure_env_name,
    f"{CLOUD_DIR}/serve",
    dockerfile_path="Dockerfile.vllm",
    inference_config=inference_config,
    update=True,
)

<br>

## 3. Serving

---

### 3.1. Create endpoint

Create an endpoint. This process does not provision a GPU cluster yet.


In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    IdentityConfiguration,
    ManagedIdentityConfiguration,
)

logger.info(f"===== 3. Serving =====")

t0 = time.time()

# Check if the endpoint already exists in the workspace
try:
    endpoint = ml_client.online_endpoints.get(azure_endpoint_name)
    logger.info("---Endpoint already exists---")
except:
    # Create an online endpoint if it doesn't exist

    # Define the endpoint
    endpoint = ManagedOnlineEndpoint(
        name=azure_endpoint_name,
        description=f"Test endpoint for {model.name}",
        # identity=IdentityConfiguration(
        #     type="user_assigned",
        #     user_assigned_identities=[ManagedIdentityConfiguration(resource_id=uai_id)],
        # )
        # if uai_id != ""
        # else None,
    )

# Trigger the endpoint creation
try:
    ml_client.begin_create_or_update(endpoint).wait()
    logger.info("\n---Endpoint created successfully---\n")
except Exception as err:
    raise RuntimeError(f"Endpoint creation failed. Detailed Response:\n{err}") from err

t1 = time.time()

from humanfriendly import format_timespan

timespan = format_timespan(t1 - t0)
logger.info(f"Creating Endpoint took {timespan}")

### 3.2. Create Deployment

Create a Deployment. This takes a lot of time as GPU clusters must be provisioned and the serving environment must be built.


In [ ]:
HF_TOKEN

In [ ]:
HF_TOKEN = d["config"]["HF_TOKEN"]
HF_MODEL_NAME_OR_PATH = d["config"]["HF_MODEL_NAME_OR_PATH"]

env_vars = {
    "MODEL_NAME": HF_MODEL_NAME_OR_PATH,
    # MODEL_NAME": "/models/outputs",
    "VLLM_ARGS": "--max-model-len 32768 --enable-lora --lora-modules phi4-lora=/models/outputs",
    "HUGGING_FACE_HUB_TOKEN": HF_TOKEN,
}
deployment_env_vars = {**env_vars}

In [ ]:
%%time
import time
from azure.ai.ml.entities import (    
    OnlineRequestSettings,
    CodeConfiguration,
    ManagedOnlineDeployment,
    ProbeSettings,
    Environment
)

t0 = time.time()
deployment = ManagedOnlineDeployment(
    name=azure_deployment_name,
    endpoint_name=azure_endpoint_name,
    model=model,
    model_mount_path="/models",
    instance_type=azure_serving_cluster_size,
    instance_count=1,
    environment_variables=deployment_env_vars,    
    environment=env,
    # scoring_script="score.py",
    # code_path="./src_serve",
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=2,
        request_timeout_ms=20000, 
        max_queue_wait_ms=60000
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=5,
        success_threshold=1,
        timeout=10,
        period=30,
        initial_delay=120
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=120,
    ),
)

# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    logger.info("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err
    
endpoint.traffic = {azure_deployment_name: 100}
endpoint_poller = ml_client.online_endpoints.begin_create_or_update(endpoint)

t1 = time.time()
timespan = format_timespan(t1 - t0)
logger.info(f"Creating deployment took {timespan}")

In [ ]:
endpoint_results = endpoint_poller.result()

<br>

## 4. Test

---

### 4.1. Invocation

Try calling the endpoint.


In [ ]:
from openai import OpenAI

url = os.path.join(endpoint_results.scoring_uri, "v1")
endpoint_name = (
    endpoint_results.name if azure_endpoint_name is None else azure_endpoint_name
)
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key  # You can paste [YOUR Azure ML API KEY] here
llm = OpenAI(base_url=url, api_key=primary_key)
model_path = HF_MODEL_NAME_OR_PATH
model_path = "phi4-lora"

In [ ]:
# Create your prompt
system_message = """
You are an AI assistant that helps customers find information. As an assistant, you respond to questions in a concise and unique manner.
You can use Markdown to answer simply and concisely, and add a personal touch with appropriate emojis.

Add a witty joke starting with "By the way," at the end of your response. Do not mention the customer's name in the joke part.
The joke should be related to the specific question asked.
For example, if the question is about tents, the joke should be specifically related to tents.

Use the given context to provide a more personalized response. Write each sentence on a new line:
"""
context = """
    The Alpine Explorer Tent features a detachable partition to ensure privacy, 
    numerous mesh windows and adjustable vents for ventilation, and a waterproof design. 
    It also includes a built-in gear loft for storing outdoor essentials. 
    In short, it offers a harmonious blend of privacy, comfort, and convenience, making it a second home in nature!
"""
question = "What are features of the Alpine Explorer Tent?"

user_message = f"""
Context: {context}
Question: {question}
"""

Simple API call

In [ ]:
# Simple API Call
response = llm.chat.completions.create(
    model="phi4-lora",
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=200,
)

print(response.choices[0].message.content)

Streaming API Call

In [ ]:
response = llm.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "saystem", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=200,
    stream=True,  # Stream the response
)

print("Streaming response:")
for chunk in response:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content"):
        print(delta.content, end="", flush=True)

Another method

In [ ]:
import requests

completions_url = os.path.join(endpoint_results.scoring_uri, "v1/completions")
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {primary_key}"}
data = {
    "model": model_path,
    "prompt": "San Francisco is a ",
    "max_tokens": 200,
    "temperature": 0.7,
}

response = requests.post(completions_url, headers=headers, json=data)
print(response.json())

### 4.2. LLM latency/throughput benchmarking


In [ ]:
import numpy as np
from time import perf_counter


def simple_llm_benchmark(
    llm: OpenAI,
    messages: list,
    model_path: str = "microsoft/Phi-4-mini-instruct",
    num_warmups: int = 1,
    num_infers: int = 5,
    **params: dict,
) -> dict:

    print("=== Measuring latency ===")
    print(f"model_path={model_path}, num_infers={num_infers}, params={params}")

    latencies = []
    # Warm up
    for _ in range(num_warmups):
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
    print("=== Warmup done. Start Benchmarking... ===")
    begin = time.time()
    # Timed run
    for curr_infer in range(num_infers):
        start_time = perf_counter()
        if (curr_infer % 5) == 0:
            print(f"Inferring {curr_infer}th...")
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
        latency = perf_counter() - start_time
        latencies.append(latency)
    end = time.time()

    # Compute run statistics
    duration = end - begin
    time_avg_sec = np.mean(latencies)
    time_std_sec = np.std(latencies)
    time_p95_sec = np.percentile(latencies, 95)
    time_p99_sec = np.percentile(latencies, 99)

    # Metrics
    metrics = {
        "duration": duration,
        "avg_sec": time_avg_sec,
        "std_sec": time_std_sec,
        "p95_sec": time_p95_sec,
        "p99_sec": time_p99_sec,
    }

    return metrics

In [ ]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
params = {
    "max_tokens": 100,
    "temperature": 0.5,
}

metrics = simple_llm_benchmark(
    llm, messages, model_path=model_path, num_warmups=1, num_infers=10, **params
)

In [ ]:
import pprint

pprint.pprint(metrics)

## Clean up


In [ ]:
!rm -rf {test_src_dir}

In [ ]:
ml_client.online_endpoints.begin_delete(azure_endpoint_name)